In [1]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero'

In [8]:
import pandas as pd
df = pd.read_csv('data/call_logs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   call_id         500 non-null    object 
 1   agent_id        500 non-null    object 
 2   org_id          500 non-null    object 
 3   installment_id  500 non-null    object 
 4   status          500 non-null    object 
 5   duration        500 non-null    float64
 6   created_ts      500 non-null    object 
 7   call_date       500 non-null    object 
dtypes: float64(1), object(7)
memory usage: 31.4+ KB


In [9]:
df.columns

Index(['call_id', 'agent_id', 'org_id', 'installment_id', 'status', 'duration',
       'created_ts', 'call_date'],
      dtype='object')

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    data_dir: Path
    STATUS_FILE: str
    all_schema: dict


In [11]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            data_dir = config.data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema,
        )

        return data_validation_config


In [22]:
import os
from mlProject import logger
import re

In [33]:
import pandas as pd
from datetime import datetime

class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True
            error_messages = []
            data = pd.read_csv(self.config.data_dir, dtype={'call_date': str})
            all_cols = list(data.columns)
            expected_cols = list(self.config.all_schema.keys())

            missing_cols = [col for col in expected_cols if col not in all_cols]
            if missing_cols:
                validation_status = False
                error_messages.append(f"Missing columns: {missing_cols}")

            if 'org_id' in data.columns:
                if not data['org_id'].isin(['O1', 'O2', 'O3']).all():
                    validation_status = False
                    invalid_ord_ids = data[~data['org_id'].isin(['O1', 'O2', 'O3'])]['org_id'].unique().tolist()
                    error_messages.append(f"Invalid ord_id values found: {invalid_ord_ids}")
            else:
                error_messages.append("Column 'org_id' not found.")

            if 'agent_id' in data.columns:
                valid_agents = [f"A{str(i).zfill(3)}" for i in range(1, 21)]
                if not data['agent_id'].isin(valid_agents).all():
                    validation_status = False
                    invalid_agents = data[~data['agent_id'].isin(valid_agents)]['agent_id'].unique().tolist()
                    error_messages.append(f"Invalid agent_id values found: {invalid_agents}")
            else:
                error_messages.append("Column 'agent_id' not found.")

            if 'call_date' in data.columns:
                data['call_date'] = pd.to_datetime(data['call_date'], errors='coerce')
                data['call_date'] = data['call_date'].dt.strftime('%#m/%#d/%Y')
                invalid_dates = []
                date_pattern = re.compile(r"^\d{1,2}/\d{1,2}/\d{4}$") 

                for date_str in data['call_date']:
                    if not isinstance(date_str, str) or not date_pattern.fullmatch(date_str.strip()):
                        invalid_dates.append(str(date_str))

                if invalid_dates:
                    validation_status = False
                    error_messages.append(f"Invalid call_date formats (expected M/D/YYYY): {invalid_dates}")
            else:
                error_messages.append("Column 'call_date' not found.")

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}\n")
                if error_messages:
                    for msg in error_messages:
                        f.write(f"{msg}\n")

            return validation_status

        except Exception as e:
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation failed due to unexpected error: {str(e)}\n")
            raise e


In [32]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-05-03 17:32:25,135: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-03 17:32:25,141: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-03 17:32:25,149: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-03 17:32:25,158: INFO: common: created directory at: artifacts]
[2025-05-03 17:32:25,160: INFO: common: created directory at: artifacts/data_validation]
